In [73]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA, LLMChain
from langchain_openai import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
import gradio as gr
from langchain.prompts import PromptTemplate
import os
from dotenv import load_dotenv


In [ ]:
dotenv_path = r"C:\Users\udhay\Downloads\.env"
load_dotenv(dotenv_path)
openai_key = os.getenv("OPENAI_API_KEY")
if not openai_key:
    raise ValueError("OPENAI_API_KEY not found in the .env file!")

os.environ["OPENAI_API_KEY"] = openai_key

In [ ]:
pdf_path = r"C:\Users\udhay\Downloads\thebook.pdf"  
loader = PyPDFLoader(pdf_path)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
print(f"Loaded and split {len(texts)} document chunks.")

Loaded and split 560 document chunks.


In [76]:
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(texts, embeddings)
retriever = vectorstore.as_retriever()

In [77]:
quiz_template = """
Based on the following context, create a quiz with 3 multiple-choice questions about {query}. 
For each question, provide 4 options and indicate the correct answer.

Context: {context}
"""

guide_template = """
Based on the following context, create a concise study guide about {query}.
Include key definitions, main concepts, and important relationships.
Format it with clear sections and bullet points for readability.

Context: {context}
"""

flashcard_template = """
Based on the following context, create 5 flashcards about {query}.
Each flashcard should have a question on one side and the answer on the other.
Format as:

Q: [Question]
A: [Answer]

Context: {context}
"""

practice_template = """
Based on the following context, create 2 practice problems about {query}.
These should be application-based problems that test understanding rather than recall.
Provide detailed solutions to each problem.

Context: {context}
"""

# Create LLM
llm = ChatOpenAI(temperature=0.7)

# Create chains for different study tools
quiz_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(template=quiz_template, input_variables=["query", "context"]),
    verbose=True
)

guide_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(template=guide_template, input_variables=["query", "context"]),
    verbose=True
)

flashcard_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(template=flashcard_template, input_variables=["query", "context"]),
    verbose=True
)

practice_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate(template=practice_template, input_variables=["query", "context"]),
    verbose=True
)

# Create standard QA chain
qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(temperature=0),
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={
        "document_variable_name": "context"
    }
)


In [78]:
def use_study_tools(input_text, tool_type="qa"):
    # Get relevant documents
    if tool_type != "qa":
        docs = retriever.get_relevant_documents(input_text)
        context = "\n\n".join([doc.page_content for doc in docs])
    
    if tool_type == "qa":
        result = qa.invoke({"query": input_text})
        return result['result']
    elif tool_type == "quiz":
        result = quiz_chain.invoke({"query": input_text, "context": context})
        return result['text']
    elif tool_type == "guide":
        result = guide_chain.invoke({"query": input_text, "context": context})
        return result['text']
    elif tool_type == "flashcard":
        result = flashcard_chain.invoke({"query": input_text, "context": context})
        return result['text']
    elif tool_type == "practice":
        result = practice_chain.invoke({"query": input_text, "context": context})
        return result['text']
    else:
        return "Invalid tool type. Choose from: qa, quiz, guide, flashcard, practice"

In [116]:
quiz_data = []
current_index = 0
user_score = 0

def get_relevant_context(query):
    docs = retriever.get_relevant_documents(query)
    context = "\n\n".join([doc.page_content for doc in docs])
    return context

def parse_quiz_questions(text):
    questions = []
    question_blocks = re.split(r'\n\s*\n', text)

    for block in question_blocks:
        if not block.strip():
            continue

        question_match = re.search(r'(?:Question\s*\d+[:.]\s*)?(.+?)(?:\n|\r\n|$)', block)
        if not question_match:
            continue

        question = question_match.group(1).strip()
        options = []
        option_matches = re.findall(r'([A-E])[).]\s*(.+?)(?:\n|\r\n|$)', block)
        if not option_matches or len(option_matches) < 5:
            continue

        for _, option_text in option_matches:
            options.append(option_text.strip())

        correct_match = re.search(r'(?:Correct\s*[Aa]nswer[:.]\s*)([A-E])', block)
        if not correct_match:
            continue

        correct_letter = correct_match.group(1)
        correct_index = ord(correct_letter) - ord('A')

        if correct_index < 0 or correct_index >= len(options):
            continue

        questions.append({
            "question": question,
            "options": options,
            "correct_letter": correct_letter,
            "correct_answer": options[correct_index]
        })

    return questions

def parse_flashcards(text):
    cards = []
    matches = re.findall(r'Q:\s*(.+?)\s*\n\s*A:\s*(.+?)(?=\n\s*Q:|$)', text, re.DOTALL)

    for question, answer in matches:
        cards.append({
            "question": question.strip(),
            "answer": answer.strip()
        })

    return cards

def generate_study_material(query, tool_type):
    if not query.strip():
        return "Please enter a topic or question."

    try:
        if tool_type == "qa":
            result = qa.invoke({"query": query})
            return result["result"]
        else:
            context = get_relevant_context(query)

            if tool_type == "quiz":
                result = quiz_chain.invoke({"query": query, "context": context})
                global quiz_data, user_score
                user_score = 0
                quiz_data = parse_quiz_questions(result["text"])
                return display_quiz_question(0)

            elif tool_type == "guide":
                result = guide_chain.invoke({"query": query, "context": context})
                return result["text"]

            elif tool_type == "flashcard":
                result = flashcard_chain.invoke({"query": query, "context": context})
                return format_flashcards(parse_flashcards(result["text"]))

            elif tool_type == "practice":
                result = practice_chain.invoke({"query": query, "context": context})
                return result["text"]

            else:
                return "Invalid tool type selected."

    except Exception as e:
        return f"An error occurred: {str(e)}"

def format_flashcards(flashcards):
    if not flashcards:
        return "No flashcards could be generated for this topic."

    formatted_text = "# Flashcards\n\n"
    for i, card in enumerate(flashcards, 1):
        formatted_text += f"### Card {i}\n"
        formatted_text += f"*Question:* {card['question']}\n\n"
        formatted_text += f"<details><summary>Click to see answer</summary>\n"
        formatted_text += f"{card['answer']}\n</details>\n\n"

    return formatted_text

def display_quiz_question(index):
    global current_question_index
    current_question_index = index

    if not quiz_data:
        return "No quiz questions available. Please generate a quiz first."

    if index >= len(quiz_data):
        return f"Quiz completed! Your score: {user_score}/{len(quiz_data)}"

    question = quiz_data[index]
    question_text = f"## Question {index + 1} of {len(quiz_data)}\n\n"
    question_text += f"{question['question']}\n\n"

    for i, option in enumerate(question['options']):
        option_letter = chr(65 + i)
        question_text += f"{option_letter}. {option}\n"

    return question_text

def check_answer(answer_letter):
    global user_score, current_question_index

    if not quiz_data or current_question_index >= len(quiz_data):
        return "No active quiz question."

    current_question = quiz_data[current_question_index]
    correct_letter = current_question["correct_letter"]

    result = f"## Question {current_question_index + 1} Result\n\n"

    if answer_letter == correct_letter:
        user_score += 1
        result += "✓ Correct!\n\n"
    else:
        result += f"✗ Incorrect. The correct answer is {correct_letter}.\n\n"

    current_question_index += 1

    if current_question_index < len(quiz_data):
        result += display_quiz_question(current_question_index)
    else:
        result += f"## Quiz Completed!\n\nYour final score: {user_score}/{len(quiz_data)}"

    return result

def reset_quiz():
    global quiz_data, current_question_index, user_score
    quiz_data = []
    current_question_index = 0
    user_score = 0
    return "Quiz reset. Ready to generate a new quiz."

with gr.Blocks(theme=gr.themes.Soft(primary_hue="indigo")) as demo:
    gr.Markdown("""
    # 📘 Interactive Study Assistant

    Ask questions, generate quizzes, study guides, flashcards, and practice problems from the machine learning book.
    """)

    with gr.Tab("Study Tools"):
        with gr.Row():
            with gr.Column(scale=3):
                input_text = gr.Textbox(
                    label="Enter your topic or question",
                    placeholder="e.g., Gradient Descent, Neural Networks, Support Vector Machines",
                    lines=3
                )
                tool_selector = gr.Radio(
                    choices=["qa", "guide", "flashcard", "practice"],
                    value="qa",
                    label="Select Study Tool",
                    info="Choose the type of study material you want to generate"
                )
                with gr.Row():
                    generate_btn = gr.Button("Generate", variant="primary")
                    clear_btn = gr.Button("Clear")

            with gr.Column(scale=4):
                output_text = gr.Markdown(label="Generated Content")

    with gr.Tab("Quiz"):
        with gr.Row():
            with gr.Column(scale=1):
                quiz_topic = gr.Textbox(
                    label="Quiz Topic",
                    placeholder="Enter a topic for your quiz",
                    lines=2
                )
                quiz_btn = gr.Button("Generate Quiz", variant="primary")
                reset_btn = gr.Button("Reset Quiz")

            with gr.Column(scale=2):
                quiz_display = gr.Markdown(label="Quiz Question")

        with gr.Row():
            answer_options = gr.Radio(
                choices=["A", "B", "C", "D", "E"],
                label="Your Answer"
            )
            submit_answer_btn = gr.Button("Submit Answer", variant="secondary")

    with gr.Tab("About"):
        gr.Markdown("""
        ## About This Tool

        This interactive study assistant helps you learn machine learning concepts from the textbook. You can:

        - *Ask questions* about specific concepts
        - *Generate quizzes* to test your knowledge
        - *Create study guides* for revision
        - *Make flashcards* for active recall practice
        - *Get practice problems* to apply your learning

        Use the different tabs to access each feature.
        """)

    generate_btn.click(
        fn=generate_study_material,
        inputs=[input_text, tool_selector],
        outputs=output_text
    )

    clear_btn.click(
        fn=lambda: "",
        inputs=None,
        outputs=output_text
    )

    quiz_btn.click(
        fn=lambda topic: generate_study_material(topic, "quiz"),
        inputs=quiz_topic,
        outputs=quiz_display
    )

    reset_btn.click(
        fn=reset_quiz,
        inputs=None,
        outputs=quiz_display
    )

    submit_answer_btn.click(
        fn=check_answer,
        inputs=answer_options,
        outputs=quiz_display
    )

demo.launch(share=True, debug=True)


* Running on local URL:  http://127.0.0.1:7865
* Running on public URL: https://bbfa7916a87b45fcf6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)




> Entering new LLMChain chain...
Prompt after formatting:

Based on the following context, create a concise study guide about linear regression.
Include key definitions, main concepts, and important relationships.
Format it with clear sections and bullet points for readability.

Context: 5.2 Regression 151
To minimize the above objective function we ﬁrst compute the gradient.
∇J(θ) = θ+
m∑
i=1
exp
(⣨
−yiˆφ(xi),θ
⟩)
1 + exp
(⣨
−yiˆφ(xi),θ
⟩)(−yiˆφ(xi))
= θ+
m∑
i=1
(p(yi|xi,θ) −1)yiˆφ(xi).
Notice that the second term of the gradient vanishes whenever p(yi|xi,θ) =
1. Therefore, one way to interpret logistic regression is to view it as a method
to maximize p(yi|xi,θ) for each point ( xi,yi) in the training set. Since the
objective function of logistic regression is twice diﬀerentiable one can also
compute its Hessian
∇2J(θ) = I−
m∑
i=1
p(yi|xi,θ)(1 −p(yi|xi,θ))ˆφ(xi)ˆφ(xi)⊤,
where we used y2
i = 1. The Hessian can be used in the Newton method
(Section 3.2.6) to obtain the optimal paramet

In [ ]:
print("Testing quiz functionality...")
test_result = use_study_tools("Gradient Descent", "quiz")
print(test_result)

Testing quiz functionality...


C:\Users\udhay\AppData\Local\Temp\ipykernel_5652\3307634288.py:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(input_text)




> Entering new LLMChain chain...
Prompt after formatting:

Based on the following context, create a quiz with 3 multiple-choice questions about Gradient Descent. 
For each question, provide 4 options and indicate the correct answer.

Context: learning applications, coordinate descent is often used because a) the cost
per iteration is very low and b) the speed of convergence may be acceptable
especially if the variables are loosely coupled.
3.2.3 Gradient Descent
Gradient descent (also widely known as steepest descent) is an optimization
technique for minimizing multidimensional smooth convex objective func-
tions of the form J : Rn →R. The basic idea is as follows: Given a location

136 3 Optimization
Stochastic gradient-based methods, by contrast, work with gradient esti-
mates obtained from small subsamples (mini-batches) of training data. This
can greatly reduce computational requirements: on large, redundant data
sets, simple stochastic gradient descent routinely outperforms soph